In [1]:
##Hoping to remove reliance on ArcGIS for ground cover processing

# This tool will not ensure that pixels and boundaries align
# That is the job of the user
import pandas
import numpy
from osgeo import gdal, gdalconst
from osgeo import ogr
import glob
import os
import datetime
import calendar
import datetime
#import matplotlib.pyplot as plt

#requiredStartDate = datetime.date(1990,1,1)
#requiredEndDate = datetime.date(1992,12,31)

#Path to shapefile to use for clip/mask
#pathToShapefile = 'D:/FractionalCover/MackayWhit_Buffer_10km_Albers.shp'
pathToShapefile = 'P:/projects/QWMN/ModelImprovementTWG/Task2.3.1_PeakRunoff/Brigalow_FI_Catchs.shp'

#Do one raster only for now
#rasterPathName = 'D:/FractionalCover/GroundCover/lztmre_qld_m198909198911_dixa2.tif'
#Do one raster only for now
#patchPathName = 'D:/FractionalCover/Patches/lztmre_rreef_m198909198911_dj4a2.tif'

#fractionalCoverDir = r'\\nrm02002\EXCHANGE\Fractional Ground Cover\Ground Cover'
#patchDir = r'\\nrm02002\EXCHANGE\Fractional Ground Cover\Ground Cover Patch'
fractionalCoverDir = r'\\nrm02002\I\Exchange\Fractional Ground Cover\Ground Cover'
patchDir = r'\\nrm02002\I\Exchange\Fractional Ground Cover\Ground Cover Patch'
#fractionalCoverDir = r'P:\projects\QWMN\ModelImprovementTWG\Task2.3.1_PeakRunoff\QLDRasters\Ground Cover'
#patchDir = r'P:\projects\QWMN\ModelImprovementTWG\Task2.3.1_PeakRunoff\QLDRasters\Ground Cover Patch'
#fractionalCoverDir = r'D:\FractionalCover\GroundCover'
#patchDir = r'D:\FractionalCover\Patches'

#outputDir = r'D:\FractionalCover\Outputs'
outputDir = r'P:\projects\QWMN\ModelImprovementTWG\Task2.3.1_PeakRunoff\BrigalowCFactors'

#Bad rasters before Sept 1987
#requiredStartDate = datetime.date(1986,6,1)
#requiredStartDate = datetime.date(1987,9,1)
requiredStartDate = datetime.date(2017,3,1)
requiredEndDate = datetime.date(2018,8,31)

#Use these to control the saved files
saveCover = False
saveCFact = True

extensions = ('*.tif', '*.tiff') # the tuple of file types

availCoverRasters = []
availPatchRasters = []

for theExt in extensions:
    ###
    availCoverRasters.extend(glob.glob(fractionalCoverDir + "/" + theExt))
    availPatchRasters.extend(glob.glob(patchDir + "/" + theExt))

#availCoverRasters = glob.glob(fractionalCoverDir + "/*.tif")
#availPatchRasters = glob.glob(patchDir)

#Final NoData Val: can't use 0, as that is a 'valid' cover val....
ourNoData = 240


In [2]:
#Open shapefile
clipPolys = ogr.Open(pathToShapefile)
lyr = clipPolys.GetLayer(0)
clipExt = lyr.GetExtent()

#Print feature count
#lyr.GetFeatureCount()

In [3]:
#print(availCoverRasters)

In [4]:
#print(availPatchRasters)

In [5]:
#define a function that will interpret a raster extent matched to the shapefil bounding box
def shapeExtent_To_Raster_Box(shapeExtent, rasterGeoInfo):
    
    #Do a bunch of stuff
    originX = rasterGeoInfo[0]
    originY = rasterGeoInfo[3]
    
    pixelWidth = rasterGeoInfo[1]
    pixelHieght = rasterGeoInfo[5]
    
    clipXMin = int((shapeExtent[0] - originX) / pixelWidth)
    clipXMax = int((shapeExtent[1] - originX) / pixelWidth) + 1
    
    clipYMin = int((shapeExtent[3] - originY) / pixelHieght)
    clipYMax = int((shapeExtent[2] - originY) / pixelHieght) + 1
    
    colCount = clipXMax - clipXMin
    rowCount = clipYMax - clipYMin
    
    newLeftCoord = originX + (clipXMin * pixelWidth)
    newTopCoord = originY + (clipYMin * pixelHieght)
    
    #This is not a Geotrans object, because I also want to return the row & col count
    #and the explicit row & col numbers to be used to clip the source raster
    newGeoInfo = (newLeftCoord, newTopCoord, pixelWidth, pixelHieght, colCount, rowCount, clipXMin, clipXMax, clipYMin, clipYMax)
    
    return newGeoInfo

In [6]:
def processCoverFile(fullFileName):
    
    fName = os.path.basename(fullFileName)
    
    beginningIdx = fName.index('_m') + 2
    endingIdx = beginningIdx + 12
    #print("Found at: " + str(beginningIdx))
    #print("Found at: " + fName[beginningIdx:endingIdx])
    
    searchBit = fName[beginningIdx:endingIdx]
    
    #print("Search bit: " + searchBit)
    
    theStartDate = datetime.date(int(fName[beginningIdx:beginningIdx+4]),int(fName[beginningIdx+4:beginningIdx+6]),1)
    
    endYear = fName[beginningIdx+6:beginningIdx+10]
    endMonth = fName[beginningIdx+10:beginningIdx+12]
    
    maxDay = calendar.monthrange(int(endYear), int(endMonth))[1]
    
    theEndDate = datetime.date(int(endYear),int(endMonth),maxDay)
    
    if (((theStartDate >= requiredStartDate) and (theStartDate <= requiredEndDate)) or ((theEndDate >= requiredStartDate) and (theEndDate <= requiredEndDate))):
        #Yep, process this fellow
        print("Processing cover for period: " + str(theStartDate) + " to " + str(theEndDate))
        
        #open cover raster
        print("Opening fractional cover: " + fName)
        #print("Opening fractional cover: " + fullFileName)
        origRas = gdal.Open(fullFileName)
        #print(origRas)
        #print(type(origRas.type))
        origRas_GeoTrans = origRas.GetGeoTransform()
        newRasterInfo = shapeExtent_To_Raster_Box(clipExt, origRas_GeoTrans)
        
        print("Working the mag...")
        sourceRast = origRas.GetRasterBand(1).ReadAsArray()

        #clip = rast[ulY:lrY, ulX:lrX]
        clipSourceRast = sourceRast[newRasterInfo[8]:newRasterInfo[9],newRasterInfo[6]:newRasterInfo[7]]

        #Introduce mask of NoDatas
        #This will include ocean as well as gaps
        #clipSourceRast = numpy.ma.masked_values(clipSourceRast, origRas.GetRasterBand(1).GetNoDataValue())
        #clipPatchRast = numpy.ma.masked_values(clipPatchRast, origPatchRas.GetRasterBand(1).GetNoDataValue())
        clipSourceRast = numpy.ma.masked_where((clipSourceRast < 100) | (clipSourceRast > 200)
                                               | (clipSourceRast ==  origRas.GetRasterBand(1).GetNoDataValue()), clipSourceRast)
        
        #Use un-patched initially
        #patchedMask = numpy.array(clipSourceRast, dtype='f')
        patchedMask = clipSourceRast
        
        #Free up some memory
        sourceRast = None
        
        #find & open patch raster
        patchFileName = None
        havePatch = False
        for patchFile in availPatchRasters:
            if(os.path.basename(patchFile).find(searchBit) != -1):
                patchFileName = patchFile
                havePatch = True
                break
        
        patchRasterInfo = None
        origPatchRas = None
        patchRast = None
        clipPatchRast = None
        clipPatchRast = None
        insertedPatch = None
        
        if havePatch:
            patchFName = os.path.basename(patchFileName)
            print("Opening patch data: " + os.path.basename(patchFName))
            origPatchRas = gdal.Open(patchFileName)
            patchRasterInfo = shapeExtent_To_Raster_Box(clipExt, origPatchRas.GetGeoTransform())
            patchRast = origPatchRas.GetRasterBand(1).ReadAsArray()
            clipPatchRast = patchRast[patchRasterInfo[8]:patchRasterInfo[9],patchRasterInfo[6]:patchRasterInfo[7]]
            clipPatchRast = numpy.ma.masked_where((clipPatchRast < 100) | (clipPatchRast > 200)
                                                  | (clipPatchRast == origPatchRas.GetRasterBand(1).GetNoDataValue()), clipPatchRast)
            patchRast = None
            clipPatchRast = numpy.ma.masked_values(clipPatchRast, origPatchRas.GetRasterBand(1).GetNoDataValue())
            #This is where we swicth in the patched data
            insertedPatch = numpy.ma.masked_values(clipSourceRast.filled(clipPatchRast), origPatchRas.GetRasterBand(1).GetNoDataValue())
            #Need to change data type of array so that we can go above values of 255
            patchedMask = numpy.array(insertedPatch, dtype='f')
            patchedMask = numpy.ma.masked_values(patchedMask, origPatchRas.GetRasterBand(1).GetNoDataValue())
        

        ###Need a 'No Data Val' that is different to Zero, because that will cause dramas for zero-cover...
        #Can't use -9999!
        patchedMask = patchedMask.filled(ourNoData)
        patchedMask = numpy.ma.masked_values(patchedMask, ourNoData)



        #Important! Use 200.0, not '200', to ensure we retain 'float' data type
        gCov = 200.0-patchedMask
        
        visPatched = 0.00925 * numpy.power(gCov, 2)

        #This essential to allow us insert NoDatas in final product... again....
        visPatched = visPatched.filled(ourNoData)
        visPatched = numpy.ma.masked_values(visPatched, ourNoData)
        
        #Cfactor....
        cFactor = numpy.exp(-0.799 - (0.0474 * visPatched) + (0.000449 * numpy.power(visPatched, 2)) - (0.0000052 * numpy.power(visPatched, 3)))
        cFactor = numpy.ma.masked_values(cFactor, ourNoData)
        
        if (saveCover):
            ###
            outputFile = os.path.join(outputDir, "cov_" + searchBit + ".tif")
            
            print("Saving: " + outputFile)

            #Integer will be fine for cover bands
            #GDT_Byte is 8 bit unsigned (apparently)
            outDS = gdal.GetDriverByName("GTiff").Create(outputFile, newRasterInfo[4], newRasterInfo[5], 1, gdal.GDT_Byte)
            outDS.SetGeoTransform((newRasterInfo[0], newRasterInfo[2], 0, newRasterInfo[1], 0, newRasterInfo[3]))
            
            #outDS.SetProjection(origRas.GetProjection())
            outBand = outDS.GetRasterBand(1)

            # write the data
            outBand.WriteArray(visPatched)

            # flush data to disk, set the NoData value and calculate stats
            outBand.FlushCache()
            outBand.SetNoDataValue(ourNoData)
            outDS.SetProjection(origRas.GetProjection())
            
            del outDS, outBand

        if (saveCFact):
            ##
            outputCFactFile = os.path.join(outputDir, "cf_" + searchBit + ".tif")
            
            print("Saving: " + outputCFactFile)
            
            outDSCFact = gdal.GetDriverByName("GTiff").Create(outputCFactFile, newRasterInfo[4], newRasterInfo[5], 1, gdal.GDT_Float32)
            outDSCFact.SetGeoTransform((newRasterInfo[0], newRasterInfo[2], 0, newRasterInfo[1], 0, newRasterInfo[3]))

            outDSCFact.SetProjection(origRas.GetProjection())
            outBandCFact = outDSCFact.GetRasterBand(1)

            outBandCFact.WriteArray(cFactor)
            outBandCFact.FlushCache()
            outBandCFact.SetNoDataValue(ourNoData)

            # georeference the image and set the projection (ROB: don't do GeoTransform, we've already done that)
            ##outDs.SetGeoTransform(inDs.GetGeoTransform())
            outDSCFact.SetProjection(origRas.GetProjection())

            #Without this 'del', the output file doesn't get populated with values???
            del outDSCFact, outBandCFact

            
            
        clipPatchRast = None
        clipSourceRast = None
        patchedMask = None
        gCov = None
        visPatched = None
        cFactor = None
        

In [7]:
for theFile in availCoverRasters:
    processCoverFile(theFile)

#print("All done")
print('Finished at ' + str(datetime.datetime.now()))

Processing cover for period: 2017-03-01 to 2017-05-31
Opening fractional cover: lztmre_qld_m201703201705_dixa2_vinterim.tif
Working the mag...
Saving: P:\projects\QWMN\ModelImprovementTWG\Task2.3.1_PeakRunoff\BrigalowCFactors\cf_201703201705.tif
Processing cover for period: 2017-06-01 to 2017-08-31
Opening fractional cover: lztmre_qld_m201706201708_dixa2_vinterim.tif
Working the mag...
Opening patch data: lztmre_qld_m201706201708_dj4a2_vinterim.tif
Saving: P:\projects\QWMN\ModelImprovementTWG\Task2.3.1_PeakRunoff\BrigalowCFactors\cf_201706201708.tif
Processing cover for period: 2017-09-01 to 2017-11-30
Opening fractional cover: lztmre_qld_m201709201711_dixa2_vinterim.tif
Working the mag...
Opening patch data: lztmre_qld_m201709201711_dj4a2_vinterim.tif
Saving: P:\projects\QWMN\ModelImprovementTWG\Task2.3.1_PeakRunoff\BrigalowCFactors\cf_201709201711.tif
Processing cover for period: 2017-12-01 to 2018-02-28
Opening fractional cover: lztmre_qld_m201712201802_dixa2_vinterim.tif
Working th